### Import Libraries

In [ ]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 100)

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

import sklearn

from sklearn.linear_model import Lasso, Ridge, ElasticNet

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.exceptions import NotFittedError

from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
# Import california housing dataset
from sklearn.datasets import fetch_california_housing
california_housing = fetch_california_housing(as_frame=True)

In [ ]:
#print(california_housing.DESCR)

In [ ]:
#Overview of entire dataset
california_housing.frame.head()

In [ ]:
# Shape of the data
california_housing.data.shape

In [ ]:
california_housing.data.head()

In [ ]:
# Summary of all features
california_housing.data.describe()

In [ ]:
# Check if the data has any missing value
california_housing.data.info()

In [ ]:
# Distributions of data features/target variable to see any potential outliers or sparse data or features that should be binary
california_housing.frame.hist(figsize=(12,10), bins = 30)
plt.subplots_adjust(hspace=0.6, wspace=0.3)
plt.show()

#### 1. Median income is more or less normally distributed but there are some people with high salary (long tail).
#### 2. House age looks more or less uniform. 
#### 3. Med House Value (target variable) has a long tail like median income. However, it's certain we have a threshold effect where all houses with a price above 5 are given the value of 5. 
#### 4. For avg rooms, avg bedrooms, avg occupation, and population the range looks large indicating there are very high and few values. 

In [ ]:
# Looking at the statistics for these features
large_range_features = ["AveRooms", "AveBedrms", "AveOccup", "Population"]
california_housing.frame[large_range_features].describe()

#### Looking at this statistics where there is a huge difference between 75% and 'max' data it seems like there are a few extreme values. 

In [ ]:
# Scatter plot of latitude and longitude data with hous value
plt.figure(figsize=(12, 10))
scatter_plot = sns.scatterplot(data=california_housing.frame,
                              x="Longitude",
                              y="Latitude",
                              size="MedHouseVal",
                              hue="MedHouseVal",
                              palette="viridis",
                              #sizes=(10,200),
                              alpha=0.5
                              )
plt.title('California Housing Prices by Location', fontsize=16)
plt.xlabel("Longitude", fontsize=14)
plt.ylabel("Latitude", fontsize=14)
plt.show()

#### All high-valued houses are be located on the coast, where the big cities from California are located: San Diego, Los Angeles, San Jose, or San Francisco.

In [ ]:
### Pair plot of all features (except latitude and longitude) against target variable 

data_filtered = california_housing.frame.drop(columns=["Longitude", "Latitude"])

data_filtered["MedHouseVal"] = pd.qcut(data_filtered["MedHouseVal"], 6, retbins=False)
data_filtered["MedHouseVal"] = data_filtered["MedHouseVal"].apply(lambda x: x.mid)

sns.pairplot(data_filtered, diag_kind="kde", hue="MedHouseVal", palette="viridis", corner=False, plot_kws={"alpha": 0.5})

plt.suptitle("Pair Plot of California Housing Features and Target", y=1.02, fontsize=16)
plt.show()

In [ ]:
### Heatmap of correlations between features
correlations = california_housing.frame.corr()

In [ ]:
correlations

In [ ]:
plt.figure(figsize=(7, 6))

sns.heatmap(correlations, cmap='RdBu', annot=True, fmt='.1f')
plt.show()

### Median income could be helpful in separating low-valued houses from high-valued houses. 

### Model Training

In [ ]:
### Create separate object for target variable
y = california_housing.frame.MedHouseVal

### Create separate object for input features
X = california_housing.frame.drop('MedHouseVal', axis=1)

In [ ]:
### Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [ ]:
### Confirmation if split was correct
### print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
### Creating a pipelines dictionary
pipelines = {
    'lasso': make_pipeline(StandardScaler(), Lasso(random_state=123)),
    'ridge': make_pipeline(StandardScaler(), Ridge(random_state=123)),
    'enet': make_pipeline(StandardScaler(), ElasticNet(random_state=123)),
    'rf': make_pipeline(StandardScaler(), RandomForestRegressor(random_state=123)),
    'gb': make_pipeline(StandardScaler(), GradientBoostingRegressor(random_state=123)),
    'xgb': make_pipeline(StandardScaler(), XGBRegressor(random_state=123))
}

In [ ]:
### Confirm that pipelines dictionary was created
### pipelines.items()

In [ ]:
### List all hyperparameters for each model
for model_name, model in pipelines.items():
    print(f"Hyperparameters for {model_name}:")
    print(model.get_params())
    print("-"*40)

In [ ]:
### Create a hyperparameter dictionary for each algorithm
hyperparameter_dict = {
    'lasso': {
        'lasso__alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]
    },
    'ridge': {
        'ridge__alpha': [0.1, 1, 10, 100, 1000]
    },
    'enet': {
        'elasticnet__alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 1, 5, 10],
        'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
    },
    'rf': {
        'randomforestregressor__n_estimators': [50, 100, 200, 300],
        'randomforestregressor__max_features': ['auto', 'sqrt', 0.33],
        'randomforestregressor__max_depth': [10, 20, 30, None],
        'randomforestregressor__min_samples_split': [2, 5, 10],
        'randomforestregressor__min_samples_leaf': [1, 2, 4]
    },
    'gb': {
        'gradientboostingregressor__n_estimators': [100, 200, 300],
        'gradientboostingregressor__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'gradientboostingregressor__max_depth': [1, 3, 5, 6],
        'gradientboostingregressor__subsample': [0.8, 0.9, 1.0]
    },
    'xgb': {
        'xgbregressor__n_estimators': [100, 200, 300],
        'xgbregressor__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'xgbregressor__max_depth': [3, 4, 5, 6],
        'xgbregressor__subsample': [0.8, 0.9, 1.0],
        'xgbregressor__colsample_bytree': [0.8, 0.9, 1.0]
    }
}

### Displaying hyperparameter dictionary
hyperparameter_dict

In [ ]:
### Checking if hyperparameters dict is setup correctly and the keys are same as pipelines dict
for key in ['lasso', 'ridge', 'enet', 'rf', 'gb', 'xgb']:
    if key in hyperparameter_dict:
        if type(hyperparameter_dict[key]) is dict:
            print(key, 'was found in hyperparameters_dict, and it is a grid')
        else:
            print(key, 'was found in hyperparameters_dict, but it is not a grid')
    else:
        print(key, 'was not found in hyperparameters_dict')

In [ ]:
### Creating a dictionary of models to store models tuned using cross-validation
fitted_models = {}

### Looping through the pipeline tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    model = GridSearchCV(pipeline, hyperparameter_dict[name], cv=10, n_jobs=-1)
    
    # Fitting model to X_train, y_train
    model.fit(X_train, y_train)
    
    # Storing the model in fitted_models dictionary
    fitted_models[name] = model
    
    # print '{name}' has been fitted
    print(name, 'has been fitted')

In [ ]:
### Checking if fitted models are of correct type by displaying its key and class of its value
for key, value in fitted_models.items():
    print(key, type(value))

In [ ]:
### Checking if models have been fitted correctly
for name, model in fitted_models.items():
    try:
        pred = model.predict(X_test)
        print(name, 'has been fitted')
    except NotFittedError as e:
        print(repr(e))

### Model Selection

In [ ]:
### Displaying the cross-validated training performance for each model in fitted_models
for name, model in fitted_models.items():
    print(name, model.best_score_)

In [ ]:
### ### Displaying the best hyperparameters for each model in fitted_models
for name, model in fitted_models.items():
    print(name, model.best_params_)

In [ ]:
### Testing the performance of each model on the test set
for name, model in fitted_models.items():
    pred = model.predict(X_test)
    print(name)
    print('------')
    print('R^2', r2_score(y_test, pred))
    print('MAE', mean_absolute_error(y_test, pred))
    print()

In [ ]:
### Plotting the performance of the winning model on the test set
winning_model_name = 'xgb'

winning_model = fitted_models[winning_model_name]

winning_model_predictions = winning_model.predict(X_test)

plt.figure(figsize=(8, 6))
plt.scatter(winning_model_predictions, y_test, alpha=0.5)
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.axline((0, 0), slope=1, color='red', linestyle='--', linewidth=2, label='Perfect Prediction')
plt.grid(True)
plt.show()


In [ ]:
### Saving the winning model
import pickle

with open('California_housing_model.pkl', 'wb') as f:
    pickle.dump(fitted_models['rf'].best_estimator_, f)